# LLM-Symbolic Explanation Refinement

Given a premise sentence $p_i$, a hypothesis sentence $h_i$, and an explanation $E_i$ consisting of a set of facts $\{f_1,f_2,...,f_n\}$, the explanation $E_i$ is considered logically valid if and only if the entailment $p_i \cup E_i \models h_i$ holds. This entailment is considered verifiable if $\{p_i, E_i, h_i\}$ can be translated into a set of logical formulae $\Phi$ that compose a theory $\Theta$. The validity of the theory $\Theta$ is subsequently determined by a theorem prover, verifying whether $\Theta \vDash \psi$, where $\psi$ represents a logical consequence derived from the logical form of $h_i$.

<img src="figures/nesy_refinement.png">

<img src="figures/nesy_refinement_results.png">

## Explanation Refinement (e-SNLI)

Premise: A couple playing with a little boy on the beach.

Hypothesis: "A couple are playing with a young child outside.

Annotated Explanation: little boy is a young child.

In [1]:
import nest_asyncio

nest_asyncio.apply()

# this also required by jupyter to start Isabelle server
import os

original_path = os.environ.get('PATH', '')
new_path = original_path + ':/workspace/Isabelle2023/bin'
os.environ['PATH'] = new_path
print(os.environ['PATH'])

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/workspace/Isabelle2023/bin


In [2]:
from refinement.refinement_model import RefinementModel
from generation.gpt import GPT
from critique import IsabelleCritique
from critique import CoherenceCritique
from critique import ParsimonyCritique
from critique import UncertaintyCritique
import yaml

from transformers.utils import logging
logging.set_verbosity(logging.CRITICAL)

# data name is default to example_data
# premise and explanation are default to None
data_name = 'esnli_0'
premise = "A couple playing with a little boy on the beach."
hypothesis = "A couple are playing with a young child outside."
explanation = "little boy is a young child."

with open('config.yaml', 'r') as file:
     config = yaml.safe_load(file)
     api_key = config.get('gpt-4o', {}).get('api_key')

llm = GPT('gpt-4o', api_key)

# hard critique model
isabelle_solver = IsabelleCritique(
    generative_model=llm,
    isabelle_session = 'HOL',
    theory_name = data_name
)

# soft critique model
coherence = CoherenceCritique()
parsimony = ParsimonyCritique()
uncertainty = UncertaintyCritique()

all_critique_models = {
    'hard': [isabelle_solver],
    'soft': [coherence, parsimony, uncertainty]
}

critique_mode = 'hard'
prompt_dict = {
    'generate explanation': 'get_explanation_prompt.txt',
    'refine no premise': 'refine_hard_no_premise_prompt.txt',
    'refine with premise': 'refine_hard_with_premise_prompt.txt'
}

refinement_model = RefinementModel(
    generative_model=llm,
    critique_models=all_critique_models,
    critique_mode=critique_mode,
    prompt_dict=prompt_dict
)

results = refinement_model.refine(
    hypothesis = hypothesis,
    premise = premise,
    explanation = explanation,
    data_name = data_name,
    iterations = 2
)

isabelle_solver.shutdown()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(



================= Iteration Number 0 ==================

Premise: A couple playing with a little boy on the beach.
Hypothesis: A couple are playing with a young child outside.
Explanation: 
1. little boy is a young child

-------------- Verification and Refinement -------------------

------------------ 1) Syntactic Parsing ------------------

Hypothesis Sentence: 
1. A couple are playing with a young child outside
Subject: A couple
Verb Phrase: are playing with a young child outside
- Main Verb: playing
- Auxiliary Verb: are
Prepositional Phrase (Accompaniment): with a young child
Adverbial Modifier (Location): outside

Explanation Sentence: 
1. little boy is a young child
Subject: little boy
Linking Verb: is
Subject Complement: a young child

Premise Sentence: 
1. A couple playing with a little boy on the beach
Subject: A couple
Verb Phrase: playing with a little boy on the beach
- Main Verb: playing
Prepositional Phrase (Accompaniment): with a little boy
Prepositional Phrase (Locat

In [4]:
print("hypothesis:", results["hypothesis"])
print("premise:", results["premise"])
print("\n------------------ Explanation ----------------------\n")
print(results["refined explanation"])

hypothesis: A couple are playing with a young child outside.
premise: A couple playing with a little boy on the beach.

------------------ Explanation ----------------------

A little boy is a young child.
Being on the beach implies being outside.


In [17]:
import ast

iteration1_str = results["history critique output"][1]

start_index = iteration1_str.find('{')
if start_index == -1:
    raise ValueError("can't find start index")

dict_str = iteration1_str[start_index:]

iteration1_dict = ast.literal_eval(dict_str)

proof_tactics = iteration1_dict['proof tactics']
print(proof_tactics)


['Sledgehammering...', 'cvc4 found a proof...', 'zipperposition found a proof...', 'cvc4: Try this: using assms explanation_1 explanation_2 by blast (6 ms)', 'zipperposition: Found duplicate proof', 'cvc4 found a proof...', 'vampire found a proof...', 'cvc4 found a proof...', 'e found a proof...', 'spass found a proof...', 'cvc4: Found duplicate proof', 'zipperposition found a proof...', 'cvc4: Found duplicate proof', 'e: Found duplicate proof', 'zipperposition: Found duplicate proof', 'spass: Found duplicate proof', 'vampire: Found duplicate proof', 'zipperposition found a proof...', 'zipperposition: Found duplicate proof', 'Done']
